In [2]:
!pip install pandas geopandas shapely pyproj numpy openpyxl

In [16]:

# ==============================================================
# ETL RESTAURANTES MADRID — V4.3 (LOCAL JUPYTERLAB, BASES LIMPIAS)
#   - B7 actualizado a _LIMPIO_v2 (sin filas "Todos")
#   - FIX DEFINITIVO B6: armoniza cod_barrio (B6 ordinal) vs COD_BAR (SHP)
#       => cod_barrio_city = cod_distrito*10 + cod_barrio (cuando aplica)
#   - Footfall con sjoin_nearest (rápido)
#   - Competencia cruda guardada (competition_raw)
# ==============================================================

import os, warnings
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

warnings.filterwarnings("ignore")

# --------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------
BASE_DIR = os.getcwd()
DATA_DIR = BASE_DIR
PATH_BARRIOS_SHP = os.path.join(BASE_DIR, "Barrios", "BARRIOS.shp")

OUT_CSV = os.path.join(BASE_DIR, "ML_RestaurantesMadrid_Features_Final_V4_3.csv")

CELL_SIZE_M = 150
BUFFER_250  = 250
BUFFER_500  = 500

CRS_WGS84  = "EPSG:4326"
CRS_METRIC = "EPSG:25830"

print("BASE_DIR:", BASE_DIR)
print("OUT_CSV:", OUT_CSV)

# --------------------------------------------------------------
# HELPERS
# --------------------------------------------------------------
def read_any(path):
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    elif ext in [".csv", ".txt"]:
        # nota: si tu CSV trae BOM (como B4), pandas suele leer bien igual
        return pd.read_csv(path)
    else:
        raise ValueError(f"Formato no soportado: {ext}")

def pick_col(df, options):
    for c in options:
        if c in df.columns:
            return c
    return None

def normalize_minmax(s):
    if not hasattr(s, "dropna"):
        s = pd.Series([s])
    s = pd.to_numeric(s, errors="coerce")
    if s.dropna().empty or s.max() == s.min():
        return s*0
    return (s - s.min())/(s.max() - s.min())

def build_square_grid_safe(points_list,
                           cell_size=CELL_SIZE_M,
                           pad=800,
                           max_cells=300_000,
                           guardrail=(350000,4300000,700000,4900000)):

    valid_layers = [g for g in points_list if g is not None and len(g)]
    bounds = [g.total_bounds for g in valid_layers if np.isfinite(g.total_bounds).all()]
    if not bounds:
        raise ValueError("No hay capas válidas para calcular grid.")

    arr = np.array(bounds)
    q = lambda a: (np.nanquantile(a,0.01), np.nanquantile(a,0.99))

    xmin = q(arr[:,0])[0] - pad
    ymin = q(arr[:,1])[0] - pad
    xmax = q(arr[:,2])[1] + pad
    ymax = q(arr[:,3])[1] + pad

    gxmin,gymin,gxmax,gymax = guardrail
    xmin=max(xmin,gxmin); ymin=max(ymin,gymin)
    xmax=min(xmax,gxmax); ymax=min(ymax,gymax)

    width, height = xmax-xmin, ymax-ymin
    nx, ny = int(np.ceil(width/cell_size)), int(np.ceil(height/cell_size))
    total = nx*ny

    if total > max_cells:
        new_size = int(np.ceil(np.sqrt((width*height)/max_cells)))
        print(f"⚠ Grid enorme ({total:,}). Ajustando cell_size → {new_size} m.")
        cell_size=new_size

    xs = np.arange(xmin, xmax, cell_size)
    ys = np.arange(ymin, ymax, cell_size)
    cells = [box(x,y,x+cell_size,y+cell_size) for x in xs for y in ys]

    grid = gpd.GeoDataFrame({"geometry":cells}, crs=CRS_METRIC)
    grid["grid_id"] = [f"cell_{i:06d}" for i in range(len(grid))]

    cent = grid.geometry.centroid
    grid["lat_center"] = cent.to_crs(CRS_WGS84).y
    grid["lon_center"] = cent.to_crs(CRS_WGS84).x

    print(f"🧱 Grid creado: {len(grid):,} celdas.")
    return grid

def to_gdf_latlon(df, lat_col, lon_col, key=""):
    tmp = df.copy()
    tmp[lat_col] = pd.to_numeric(tmp[lat_col], errors="coerce")
    tmp[lon_col] = pd.to_numeric(tmp[lon_col], errors="coerce")
    tmp = tmp.dropna(subset=[lat_col, lon_col])
    if tmp.empty:
        raise ValueError(f"{key}: sin coords válidas en {lat_col}/{lon_col}")
    gdf = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp[lon_col], tmp[lat_col]), crs=CRS_WGS84)
    return gdf.to_crs(CRS_METRIC)

def to_gdf_xy(df, x_col, y_col, key=""):
    tmp = df.copy()
    tmp[x_col] = pd.to_numeric(tmp[x_col], errors="coerce")
    tmp[y_col] = pd.to_numeric(tmp[y_col], errors="coerce")
    tmp = tmp.dropna(subset=[x_col, y_col])
    if tmp.empty:
        raise ValueError(f"{key}: sin coords válidas en {x_col}/{y_col}")
    gdf = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp[x_col], tmp[y_col]), crs=CRS_METRIC)
    return gdf

def count_points_within(gdf_points, centers, radius_m):
    if gdf_points is None or len(gdf_points)==0:
        return pd.Series(0, index=centers.index)
    return centers.apply(lambda c: gdf_points.within(c.buffer(radius_m)).sum())

def min_distance_to(gdf_points, centers):
    if gdf_points is None or len(gdf_points)==0:
        return pd.Series(np.nan, index=centers.index)
    return centers.apply(lambda c: gdf_points.distance(c).min())

def sum_points_within(gdf_points, centers, radius_m, value_col):
    if gdf_points is None or len(gdf_points)==0 or (value_col is None) or (value_col not in gdf_points.columns):
        return pd.Series(0.0, index=centers.index)
    return centers.apply(lambda c: gdf_points.loc[gdf_points.within(c.buffer(radius_m)), value_col].sum())

def harmonize_cod_barrio(df, col_d="cod_distrito", col_b="cod_barrio"):
    """
    Armoniza cod_barrio:
    - Si parece "ordinal" (1..9, o típicamente < 11), lo convierte a "código ciudad" = distrito*10 + barrio
    - Si ya parece "código ciudad" (>= 11, 3 dígitos, etc.), lo deja igual.
    """
    out = df.copy()
    out[col_d] = pd.to_numeric(out[col_d], errors="coerce")
    out[col_b] = pd.to_numeric(out[col_b], errors="coerce")
    # criterio práctico: si el máximo de cod_barrio es <= 10, casi seguro es ordinal
    if pd.to_numeric(out[col_b], errors="coerce").max(skipna=True) <= 10:
        out[col_b] = (out[col_d] * 10 + out[col_b])
    return out

# --------------------------------------------------------------
# CARGA BASES LIMPIAS
# --------------------------------------------------------------
paths = {
    "B1": os.path.join(DATA_DIR,"B1_Restaurantes_2025_Lat_Long_CLEAN.csv"),
    "B2": os.path.join(DATA_DIR,"B2_Menu_RestaurantesMX_LIMPIO.csv"),
    "B3": os.path.join(DATA_DIR,"B3_Restaurantes_Terrazas_2025_Lat_Long_LIMPIO.csv"),
    "B4": os.path.join(DATA_DIR,"B4_Flujo_Peatones_2024_LIMPIO.csv"),
    "B5": os.path.join(DATA_DIR,"B5_Licencias_2025_Lat_Long_LIMPIO.csv"),
    "B6": os.path.join(DATA_DIR,"B6_Residentes_Edad_Nacionalidad_2025_LIMPIO.csv"),
    "B7": os.path.join(DATA_DIR,"B7_Poblacion_Madrid_LIMPIO_v2.csv"),  # ✅ v2
    "B8": os.path.join(DATA_DIR,"B8_Aparcamientos_Publicos.xlsx"),
    "B9": os.path.join(DATA_DIR,"B9_Estaciones_Metro_Renfe_CLEAN.csv"),
}

missing = [(k,p) for k,p in paths.items() if not os.path.exists(p)]
if missing:
    print("❌ FALTAN ARCHIVOS:")
    for k,p in missing:
        print("  -", k, "->", p)
    raise FileNotFoundError("Corrige los nombres/rutas de los archivos faltantes y reintenta.")

dfs = {k: read_any(p) for k,p in paths.items()}
gdf_barrios = gpd.read_file(PATH_BARRIOS_SHP)

print("✅ Bases cargadas:", {k: dfs[k].shape for k in dfs})

# --------------------------------------------------------------
# GEO CAPAS
# --------------------------------------------------------------
gdfs = {}

# --- B1 ---
dfB1 = dfs["B1"].copy()
xB1  = pick_col(dfB1, ["x_num","coordenada_x_local","COORDENADA-X","X","x"])
yB1  = pick_col(dfB1, ["y_num","coordenada_y_local","COORDENADA-Y","Y","y"])
latB1= pick_col(dfB1, ["Latitud","lat","latitude","lat_num","Latitud_num"])
lonB1= pick_col(dfB1, ["Longitud","longitud","lon","longitude","lon_num","Longitud_num","longitud_num"])

if xB1 and yB1:
    gdfs["B1"] = to_gdf_xy(dfB1, xB1, yB1, "B1")
elif latB1 and lonB1:
    gdfs["B1"] = to_gdf_latlon(dfB1, latB1, lonB1, "B1")
else:
    raise ValueError(f"B1: no encuentro coords. Columnas: {list(dfB1.columns)}")

# --- B3 (terrazas) ---
dfB3 = dfs["B3"].copy()
xB3 = pick_col(dfB3, ["coordenada_x_local","x_num","COORDENADA-X"])
yB3 = pick_col(dfB3, ["coordenada_y_local","y_num","COORDENADA-Y"])
if xB3 and yB3:
    gdfs["B3"] = to_gdf_xy(dfB3, xB3, yB3, "B3")
else:
    latB3 = pick_col(dfB3, ["Latitud","lat"])
    lonB3 = pick_col(dfB3, ["Longitud","longitud","lon"])
    gdfs["B3"] = to_gdf_latlon(dfB3, latB3, lonB3, "B3")

# --- B4 (peatones) ---
dfB4 = dfs["B4"].copy()
lat4 = pick_col(dfB4, ["Latitud_num","Latitud","lat","latitude"])
lon4 = pick_col(dfB4, ["Longitud_num","longitud_num","Longitud","longitud","lon","longitude"])
if lat4 is None or lon4 is None:
    raise ValueError(f"B4: no encuentro lat/lon. Columnas: {list(dfB4.columns)}")
gdfs["B4"] = to_gdf_latlon(dfB4, lat4, lon4, "B4")

# --- B5 (licencias) ---
dfB5 = dfs["B5"].copy()
xB5 = pick_col(dfB5, ["x_num","coordenada_x_local","COORDENADA-X"])
yB5 = pick_col(dfB5, ["y_num","coordenada_y_local","COORDENADA-Y"])
if xB5 and yB5:
    gdfs["B5"] = to_gdf_xy(dfB5, xB5, yB5, "B5")
else:
    latB5 = pick_col(dfB5, ["Latitud","lat"])
    lonB5 = pick_col(dfB5, ["Longitud","longitud","lon"])
    gdfs["B5"] = to_gdf_latlon(dfB5, latB5, lonB5, "B5")

# --- B8 (parkings) ---
dfB8 = dfs["B8"].copy()
xB8 = pick_col(dfB8, ["COORDENADA-X","coordenada_x_local","x_num","X"])
yB8 = pick_col(dfB8, ["COORDENADA-Y","coordenada_y_local","y_num","Y"])
if xB8 and yB8:
    gdfs["B8"] = to_gdf_xy(dfB8, xB8, yB8, "B8")
else:
    latB8 = pick_col(dfB8, ["LATITUD","Latitud","lat"])
    lonB8 = pick_col(dfB8, ["LONGITUD","Longitud","longitud","lon"])
    gdfs["B8"] = to_gdf_latlon(dfB8, latB8, lonB8, "B8")

# --- B9 (metro/renfe) ---
dfB9 = dfs["B9"].copy()
latB9 = pick_col(dfB9, ["Latitud","lat"])
lonB9 = pick_col(dfB9, ["Longitud","longitud","lon"])
gdfs["B9"] = to_gdf_latlon(dfB9, latB9, lonB9, "B9")

print("✅ Capas GEO:", {k: len(gdfs[k]) for k in gdfs})

# --------------------------------------------------------------
# GRID
# --------------------------------------------------------------
grid = build_square_grid_safe([gdfs["B1"], gdfs["B5"], gdfs["B8"], gdfs["B9"], gdfs["B4"]])
cent = grid.geometry.centroid
cent_gdf = gpd.GeoDataFrame(grid[["grid_id"]].copy(), geometry=cent, crs=CRS_METRIC)

# --------------------------------------------------------------
# FEATURES BASE
# --------------------------------------------------------------
print("⚙ Features base…")

grid["rest_total_250m"] = count_points_within(gdfs["B1"], cent, BUFFER_250)
grid["rest_total_500m"] = count_points_within(gdfs["B1"], cent, BUFFER_500)
grid["rest_density_500m"] = grid["rest_total_500m"] / (np.pi * (BUFFER_500**2))

# competencia cruda (para ML)
grid["competition_raw"] = grid["rest_density_500m"]

grid["dist_metro_min"] = min_distance_to(gdfs["B9"], cent)

# Footfall (nearest 5)
foot_col = pick_col(gdfs["B4"], ["cantidad_peatones_num","cantidad_peatones"])
if foot_col is None:
    grid["footfall_day_avg"] = np.nan
else:
    nearest = gpd.sjoin_nearest(
        cent_gdf,
        gdfs["B4"][[foot_col, "geometry"]].copy(),
        how="left",
        distance_col="dist",
        max_distance=2000
    )
    nearest = nearest.sort_values(["grid_id","dist"]).groupby("grid_id").head(5)
    foot = nearest.groupby("grid_id")[foot_col].mean().rename("footfall_day_avg")
    grid = grid.merge(foot, on="grid_id", how="left")

# --------------------------------------------------------------
# INTERMODAL SCORE (metro + parking)
# --------------------------------------------------------------
print("⚙ Intermodal score…")

grid["estaciones_500m"] = count_points_within(gdfs["B9"], cent, 500)
grid["dist_parking_min"] = min_distance_to(gdfs["B8"], cent)

cap_col = pick_col(gdfs["B8"], ["Plazas_Publicas","PlazasPublicas","PLAZAS_PUBLICAS","plazas_publicas","plazas"])
grid["plazas_parking_500m"] = (
    sum_points_within(gdfs["B8"], cent, 500, cap_col) if cap_col else count_points_within(gdfs["B8"], cent, 500)
)

dist_metro_norm   = 1 - normalize_minmax(grid["dist_metro_min"])
dist_parking_norm = 1 - normalize_minmax(grid["dist_parking_min"])
estaciones_norm   = normalize_minmax(grid["estaciones_500m"])
plazas_norm       = normalize_minmax(grid["plazas_parking_500m"])

grid["intermodal_score"] = (
    0.45*dist_metro_norm +
    0.15*estaciones_norm +
    0.40*((plazas_norm + dist_parking_norm)/2.0)
).fillna(0).clip(0,1)

# --------------------------------------------------------------
# SHP NORMALIZADO (base para B7 y B6)
# --------------------------------------------------------------
gdf_barrios_25830 = gdf_barrios.to_crs(CRS_METRIC).copy()

gdf_barrios_25830["cod_distrito"] = pd.to_numeric(
    gdf_barrios_25830["CODDIS"].astype(str).str.strip(),
    errors="coerce"
).astype("Int64")

# COD_BAR viene como '011' -> 11
gdf_barrios_25830["cod_barrio"] = pd.to_numeric(
    gdf_barrios_25830["COD_BAR"].astype(str).str.strip().str.lstrip("0").replace("", "0"),
    errors="coerce"
).astype("Int64")

# --------------------------------------------------------------
# DEMANDA RESIDENTE (B7 v2) + BARRIOS
# --------------------------------------------------------------
print("⚙ Demanda residente (B7)…")

df7 = dfs["B7"].copy()
pop_col = pick_col(df7, ["num_personas_num","num_personas"])
if pop_col is None:
    raise ValueError(f"B7: no encuentro columna población. Columnas: {list(df7.columns)}")

df7["cod_distrito"] = pd.to_numeric(df7["cod_distrito"], errors="coerce")
df7["cod_barrio"]   = pd.to_numeric(df7["cod_barrio"], errors="coerce")
df7[pop_col]        = pd.to_numeric(df7[pop_col], errors="coerce")

# 🔧 armoniza si B7 trae ordinal (1..9)
df7 = harmonize_cod_barrio(df7, "cod_distrito", "cod_barrio")

df7 = df7.dropna(subset=["cod_distrito","cod_barrio"])
df7["cod_distrito"] = df7["cod_distrito"].astype(int)
df7["cod_barrio"]   = df7["cod_barrio"].astype(int)

df7_agg = df7.groupby(["cod_distrito","cod_barrio"], as_index=False)[pop_col].sum()
df7_agg = df7_agg.rename(columns={pop_col:"num_personas"})

gdf_dem = gdf_barrios_25830.merge(df7_agg, on=["cod_distrito","cod_barrio"], how="left")
gdf_dem["num_personas"] = gdf_dem["num_personas"].fillna(0)

grid_barr = gpd.sjoin(
    cent_gdf,
    gdf_dem[["cod_distrito","cod_barrio","num_personas","geometry"]],
    how="left",
    predicate="within"
)

grid = grid.merge(
    grid_barr[["grid_id","cod_distrito","cod_barrio","num_personas"]],
    on="grid_id",
    how="left"
)
grid["num_personas"] = grid["num_personas"].fillna(0)
grid["res_density"]  = normalize_minmax(grid["num_personas"]).fillna(0)

# --------------------------------------------------------------
# B6 LIMPIO — Mex affinity (FIX REAL: convertir barrio ordinal a código ciudad)
# --------------------------------------------------------------
print("⚙ Mex affinity (B6) — armonizando cod_barrio…")

df6 = dfs["B6"].copy()
df6["cod_distrito"] = pd.to_numeric(df6["cod_distrito"], errors="coerce")
df6["cod_barrio"]   = pd.to_numeric(df6["cod_barrio"], errors="coerce")
df6["mex_affinity_proxy"] = pd.to_numeric(df6["mex_affinity_proxy"], errors="coerce")

# 🔧 si B6 trae ordinal (1..9) => cod_barrio = distrito*10 + barrio
df6 = harmonize_cod_barrio(df6, "cod_distrito", "cod_barrio")

df6["cod_distrito"] = df6["cod_distrito"].astype("Int64")
df6["cod_barrio"]   = df6["cod_barrio"].astype("Int64")

# Merge B6 sobre geometría barrios
gdf_b6 = gdf_barrios_25830.merge(
    df6[["cod_distrito","cod_barrio","mex_affinity_proxy"]],
    on=["cod_distrito","cod_barrio"],
    how="left"
)

matches = gdf_b6["mex_affinity_proxy"].notna().sum()
print("Matches B6↔Barrios:", matches, "/", len(gdf_b6))

gdf_b6["mex_affinity_proxy"] = gdf_b6["mex_affinity_proxy"].fillna(0)

# Spatial join a centros de grid
grid_b6 = gpd.sjoin(
    cent_gdf,
    gdf_b6[["mex_affinity_proxy","geometry"]],
    how="left",
    predicate="within"
)

grid = grid.merge(grid_b6[["grid_id","mex_affinity_proxy"]], on="grid_id", how="left")
grid["mex_affinity_proxy"] = grid["mex_affinity_proxy"].fillna(0)

# --------------------------------------------------------------
# B2 LIMPIO: presencia MX (mapea id_local de B2 contra B1)
# --------------------------------------------------------------
print("⚙ Presencia MX (B2)…")

df2 = dfs["B2"].copy()

grid["mx_rest_total_500m"] = 0
if "id_local" in df2.columns:
    mx_ids = set(pd.to_numeric(df2["id_local"], errors="coerce").dropna().astype(int).tolist())
    idcol_b1 = pick_col(gdfs["B1"], ["id_local","id","ID_LOCAL"])
    if idcol_b1:
        b1_ids = pd.to_numeric(gdfs["B1"][idcol_b1], errors="coerce")
        gdf_mx = gdfs["B1"].loc[b1_ids.isin(mx_ids)].copy()
        grid["mx_rest_total_500m"] = count_points_within(gdf_mx, cent, 500)

# --------------------------------------------------------------
# SCORE / OUTPUTS
# --------------------------------------------------------------
ratio = grid["rest_total_250m"] / (grid["rest_total_500m"].replace(0, np.nan))
ratio = ratio.fillna(0).clip(0,1)

grid["footfall_x_mex_share"] = pd.to_numeric(grid["footfall_day_avg"], errors="coerce") * (1 - ratio)

footfall_norm = normalize_minmax(grid["footfall_day_avg"]).fillna(0)
access_norm   = normalize_minmax(grid["intermodal_score"]).fillna(0)
compet_norm   = normalize_minmax(1/(1+grid["rest_density_500m"])).fillna(0)
mx_aff_norm   = normalize_minmax(grid["mex_affinity_proxy"]).fillna(0)

grid["mexican_location_score"] = (
    0.35*footfall_norm +
    0.25*access_norm +
    0.20*compet_norm +
    0.20*mx_aff_norm
).fillna(0).clip(0,1)

# --------------------------------------------------------------
# SANITY CHECKS
# --------------------------------------------------------------
print("\n--- SANITY CHECKS ---")
print("footfall_day_avg describe:\n", pd.to_numeric(grid["footfall_day_avg"], errors="coerce").describe())
print("intermodal_score describe:\n", pd.to_numeric(grid["intermodal_score"], errors="coerce").describe())
print("mex_affinity_proxy describe:\n", pd.to_numeric(grid["mex_affinity_proxy"], errors="coerce").describe())
print("mexican_location_score describe:\n", pd.to_numeric(grid["mexican_location_score"], errors="coerce").describe())

# --------------------------------------------------------------
# EXPORT
# --------------------------------------------------------------
cols_out = [
    "grid_id","lat_center","lon_center",
    "footfall_day_avg",
    "rest_total_250m","rest_total_500m","rest_density_500m",
    "competition_raw",
    "dist_metro_min","estaciones_500m",
    "dist_parking_min","plazas_parking_500m","intermodal_score",
    "num_personas","res_density",
    "mex_affinity_proxy","mx_rest_total_500m",
    "footfall_x_mex_share",
    "mexican_location_score",
]

grid[cols_out].to_csv(OUT_CSV, index=False, encoding="utf-8")
print(f"\n✅ ETL V4.3 exportado → {OUT_CSV}")


BASE_DIR: C:\Users\geith\TFM INESDI
OUT_CSV: C:\Users\geith\TFM INESDI\ML_RestaurantesMadrid_Features_Final_V4_3.csv
✅ Bases cargadas: {'B1': (19325, 21), 'B2': (82, 8), 'B3': (6298, 19), 'B4': (128833, 8), 'B5': (59752, 19), 'B6': (131, 7), 'B7': (1064, 10), 'B8': (53, 15), 'B9': (250, 6)}
✅ Capas GEO: {'B1': 19325, 'B3': 6298, 'B4': 128833, 'B5': 59752, 'B8': 53, 'B9': 250}
🧱 Grid creado: 39,858 celdas.
⚙ Features base…
⚙ Intermodal score…
⚙ Demanda residente (B7)…
⚙ Mex affinity (B6) — armonizando cod_barrio…
Matches B6↔Barrios: 131 / 131
⚙ Presencia MX (B2)…

--- SANITY CHECKS ---
footfall_day_avg describe:
 count    5569.000000
mean      342.931693
std       396.052182
min         1.600000
25%        29.000000
50%       235.400000
75%       490.000000
max      6307.600000
Name: footfall_day_avg, dtype: float64
intermodal_score describe:
 count    39858.000000
mean         0.478252
std          0.139542
min          0.000000
25%          0.396904
50%          0.503462
75%          